<img align="left" style="padding-right:30px;" src="./Images/logo_UNSW.svg" height="100"><img align="left" style="padding-right:30px;" src="./Images/Green_RGB.png" height="90"><img align="left" style="padding-right:30px;" src="./Images/image004.png" height="100">



[![DOI](https://zenodo.org/badge/940091341.svg)](https://doi.org/10.5281/zenodo.14939868) <a href="https://www.globh2e.org.au/"><img src="https://img.shields.io/badge/ARC:Funding%20number-IC200100023-blue.svg"/></a>

# **<div style="text-align: left; font-size: 30px;"> Pathways to global hydrogen production within planetary boundaries**</div>
**<div style="text-align: left; font-size: 25px;"> Life cycle optimisation - Tutorial**</div>

<div style="text-align: left; font-size: 16px;">Michaël Lejeune<sup>a,b</sup>, Sami Kara<sup>a,b</sup>, Michael Zwicky Hauschild<sup>c,d</sup>, Sareh Sharabifarahni<sup>a</sup>, Rahman Daiyan<sup>b,e</sup></div><br>

<div style="text-align: left; font-size: 13px;"><sup>a</sup>Sustainability in Manufacturing and Life Cycle Engineering Research Group, School of Mechanical and Manufacturing Engineering, the University of New South Wales, 2052, Sydney, Australia</div>

<div style="text-align: left; font-size: 13px;">
<sup>b</sup>Australian Research Council Training Centre for the Global Hydrogen Economy (GlobH2e), the University of New South Wales, 2052, Sydney, Australia</div>

<div style="text-align: left; font-size: 13px;">
<sup>c</sup>Centre for Absolute Sustainability, Technical University of Denmark, Kgs, Lyngby, Denmark</div>

<div style="text-align: left; font-size: 13px;">
<sup>d</sup>Section for Quantitative Sustainability Assessment (QSA), Department of Environmental and Resource Engineering, Technical University of Denmark, Kgs, Lyngby, Denmark</div>

<div style="text-align: left; font-size: 13px;">
<sup>e</sup>School of Minerals and Energy Engineering, The University of New South Wales, Sydney 2052, Australia</div><br>


# Target audience & prerequisites

> <span style="color:rgba(22, 210, 69, 1); font-weight: bold;">Target audience</span><br>
>- Life cycle assessment practioners/researchers
>- Absolute environmental sustainability practioners/researchers

> <span style="color:rgba(191, 200, 30, 1); font-weight: bold;">Prerequisites</span><br>
>- Background in life cycle assessment
>- Brightway
>- Activity browser (with ScenarioLink installed)
>- Julia with JuMP.jl
>- CPLEX optimisation solver
>- IDE (e.g., VS code)

# Intro to AESA

<div style="text-align:center;">
  <img src="./Images/framwork2.svg">
</div>

<!-- # Contents <span style="color:rgba(0, 0, 0, 1); font-weight: bold;">Contents</span><br> -->
# Contents

>1. Background information on computational LCA and optimisation
>2. Installation of planetary boundaries characterisation factors
>3. Modelling aggregated inventories on Activity - browser
>4. pre-optimisation formating on Excel
>5. Implementation of the optimisation model in Julia using JuMP.jl


# **Case study**

<!-- > <span style="color:rgba(184, 193, 29, 1); font-weight: bold;">Simple optimisation example</span><br>
>- Optimisation of electrical mix for hydrogen production via PEM electrolysis
>- Using activity browser, excel and Julia (a python version is also possible)
>- Integration of prospective LCA data in the optimisation.
>- Optional - Cost and material tracking -->


> <span style="color:rgba(26, 190, 67, 1); font-weight: bold;">Goal  & Scope definition</span><br>
>- Goal: Minimising the effective planetary footprint of hydrogen production via PEM electrolysis.
>- LCA type: prospective attributional LCA (note that the appraoch can also be interpreted as consequential)
>- Functional unit: 225 MtH₂/year (2050) (global annual production of H₂ via water electrolysis)
>- Scope: Cradle to gate (electricity production + PEM electrolysis)
>- Database: ecoinvent 3.9.1 (cut-off) - updated with premise - SSP1-PkBudg500
<div style="text-align:center;">
  <img src="./Images//PEM_unit1.svg">
</div>


### **Initialisation**

In [1]:
using JuMP, CPLEX,Distributions,DataFrames,LinearAlgebra,XLSX
using PyPlot
using SparseArrays
using JLD2

using lce
initProject("Natcoms")
⊘ = ./;


┌ Info: Project Natcoms -> model:ecoinvent_3.9.1 RCP:0.0 SSP:0 year:0 loaded
└ @ lce /Users/mickael/Library/CloudStorage/OneDrive-UNSW/Research/Publications/Journal articles/1_Natcoms/code/lce/src/lce.jl:781


# Data import

In [ ]:
aSOS = XLSX.readtable("./data/Fig2c.xlsx", "allocated SOS") |> DataFrame # %SOS/GtH2/yr



Row,Year,SOS_per_GtH2_05,SOS_per_GtH2_25,SOS_per_GtH2_50,SOS_per_GtH2_75,SOS_per_GtH2_95
,Any,Any,Any,Any,Any,Any
1,2025.0,6.73849,8.91468,9.53045,13.0886,13.7765
2,2030.0,4.29387,5.98068,7.23146,9.79054,12.4713
3,2035.0,2.40314,3.25648,4.07245,7.64923,9.58434
4,2040.0,1.50013,1.98989,2.65428,4.63441,7.64559
5,2045.0,1.07436,1.51761,2.13706,2.69473,6.52992
6,2050.0,0.761957,1.18881,1.59743,2.49928,5.57836


In [3]:
𝚪ᵦ=XLSX.readtable("./data/interactions_matrices.xlsx", "𝚪ᵦ") |> DataFrame


Row,Climate change Energy imbalance,Climate change CO2 Concentration,Ocean acidification,Atmospheric aerosol loading,Freshwater use,Biogeochemical flows-P,Biogeochemical flows-N,Stratospheric ozone depletion,Land-system change,Biosphere Integrity
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1.22241,0,-0.0100732,0.0232257,-0.111239,0.232257,0.232257,-0.0710218,0.122241,0.38386
2,0,1.22241,-0.0100732,0.0232257,-0.111239,0.232257,0.232257,-0.0710218,0.122241,0.38386
3,0.325108,0,1.06115,0.00617706,-0.0295849,0.0617706,0.0617706,-0.0188888,0.0325108,0.4567
4,-0.684548,0,0.00564097,0.986994,0.0622939,-0.130064,-0.130064,0.0397722,-0.0684548,-0.214962
5,0.220814,0,0.0407336,0.00419546,0.979906,0.0419546,0.0419546,-0.0128293,0.0220814,0.416858
6,0.206199,0,0.0138327,0.103918,-0.0187641,1.03918,1.03918,-0.00198018,0.0206199,0.426595
7,0.206199,0,0.0138327,0.103918,-0.0187641,1.03918,1.03918,-0.00198018,0.0206199,0.426595
8,-0.134465,0,0.00110805,-0.00255483,0.0122363,-0.0255483,-0.0255483,1.00781,-0.0134465,-0.0422246
9,0.428032,0,0.233494,0.00813261,-0.148951,0.0813261,0.0813261,-0.0248687,1.0428,0.470084


In [ ]:
SOS = XLSX.readtable("./data/SOS.xlsx", "Planetary boundaries") |> DataFrame


Row,Control variable,Pre-industrial value,Planetary boundary,SOS,High risk,High risk space,Units
,Any,Any,Any,Any,Any,Any,Any
1,Climate change-Energy imbalance,0.0,1.0,1.0,1.5,1.5,W/m2
2,Climate change-CO2 Concentration,278.0,350.0,72.0,450.0,172.0,ppm
3,Ocean acidification-Carbonate ion concentration,3.44,2.752,0.688,2.408,1.032,Ωarag
4,Atmospheric aerosol loading-Aerosol Optical Depth (AOD),0.03,0.1,0.07,0.25,0.22,Aerosol optical depth
5,Freshwater use-Global,0.0,4000.0,4000.0,6000.0,6000.0,km3
6,Biogeochemical flows-P,0.0,11.0,11.0,100.0,100.0,TgP
7,Biogeochemical flows-N,0.0,62.0,62.0,82.0,82.0,TgN
8,Stratospheric ozone depletion-Stratospheric O3 concentration,290.0,276.0,14.0,261.0,29.0,Dobson units
9,Land-system change-Global,100.0,75.0,25.0,54.0,46.0,%


Safe operaring space 

In [6]:
categories=SOS[:,1]


10-element Vector{Any}:
 "Climate change-Energy imbalance"
 "Climate change-CO2 Concentration"
 "Ocean acidification-Carbonate ion concentration"
 "Atmospheric aerosol loading-Aerosol Optical Depth (AOD)"
 "Freshwater use-Global"
 "Biogeochemical flows-P"
 "Biogeochemical flows-N"
 "Stratospheric ozone depletion-Stratospheric O3 concentration"
 "Land-system change-Global"
 "Biosphere Integrity-Change in biosphere integrity"

In [7]:
Δ𝐗ᴾᴮ=SOS[:,4]


10-element Vector{Any}:
    1.0
   72.0
    0.6880000000000002
    0.07
 4000.0
   11.0
   62.0
   14.0
   25.0
   10.0

In [8]:
units=SOS[:,end]


10-element Vector{Any}:
 "W/m2"
 "ppm"
 "Ωarag"
 "Aerosol optical depth"
 "km3"
 "TgP"
 "TgN"
 "Dobson units"
 "%"
 "%"

Allocation factor

In [36]:
α_2050=aSOS[end,4] #%SOS/GtH2 (median value)


1.5974315890692654

We want to express it in an allocation factor per kg of H2 produced.

In [37]:
ω_2050°=α_2050/100*Δ𝐗ᴾᴮ # allocation factor per GtH2
𝛚 = ω_2050° * 1e-12 # allocation factor per kgH2/yr
[a=>b=>c for (a,b,c) ∈ zip(categories,𝛚,units)]


10-element Vector{Pair{String, Pair{Float64, String}}}:
                              "Climate change-Energy imbalance" => (1.5974315890692653e-14 => "W/m2")
                             "Climate change-CO2 Concentration" => (1.1501507441298711e-12 => "ppm")
              "Ocean acidification-Carbonate ion concentration" => (1.099032933279655e-14 => "Ωarag")
      "Atmospheric aerosol loading-Aerosol Optical Depth (AOD)" => (1.118202112348486e-15 => "Aerosol optical depth")
                                        "Freshwater use-Global" => (6.389726356277062e-11 => "km3")
                                       "Biogeochemical flows-P" => (1.757174747976192e-13 => "TgP")
                                       "Biogeochemical flows-N" => (9.904075852229445e-13 => "TgN")
 "Stratospheric ozone depletion-Stratospheric O3 concentration" => (2.2364042246969718e-13 => "Dobson units")
                                    "Land-system change-Global" => (3.9935789726731633e-13 => "%")
            

Now we compute the interaction matrix, we consider biophysical interactions only.

In [6]:
𝚪ᵦ


Row,Climate change Energy imbalance,Climate change CO2 Concentration,Ocean acidification,Atmospheric aerosol loading,Freshwater use,Biogeochemical flows-P,Biogeochemical flows-N,Stratospheric ozone depletion,Land-system change,Biosphere Integrity
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1.22241,0,-0.0100732,0.0232257,-0.111239,0.232257,0.232257,-0.0710218,0.122241,0.38386
2,0,1.22241,-0.0100732,0.0232257,-0.111239,0.232257,0.232257,-0.0710218,0.122241,0.38386
3,0.325108,0,1.06115,0.00617706,-0.0295849,0.0617706,0.0617706,-0.0188888,0.0325108,0.4567
4,-0.684548,0,0.00564097,0.986994,0.0622939,-0.130064,-0.130064,0.0397722,-0.0684548,-0.214962
5,0.220814,0,0.0407336,0.00419546,0.979906,0.0419546,0.0419546,-0.0128293,0.0220814,0.416858
6,0.206199,0,0.0138327,0.103918,-0.0187641,1.03918,1.03918,-0.00198018,0.0206199,0.426595
7,0.206199,0,0.0138327,0.103918,-0.0187641,1.03918,1.03918,-0.00198018,0.0206199,0.426595
8,-0.134465,0,0.00110805,-0.00255483,0.0122363,-0.0255483,-0.0255483,1.00781,-0.0134465,-0.0422246
9,0.428032,0,0.233494,0.00813261,-0.148951,0.0813261,0.0813261,-0.0248687,1.0428,0.470084


We just need to make it a matrix

In [13]:
𝚪=Matrix{Float64}(𝚪ᵦ)


10×10 Matrix{Float64}:
  1.22241   0.0      -0.0100732   …  -0.0710218    0.122241    0.38386
  0.0       1.22241  -0.0100732      -0.0710218    0.122241    0.38386
  0.325108  0.0       1.06115        -0.0188888    0.0325108   0.4567
 -0.684548  0.0       0.00564097      0.0397722   -0.0684548  -0.214962
  0.220814  0.0       0.0407336      -0.0128293    0.0220814   0.416858
  0.206199  0.0       0.0138327   …  -0.00198018   0.0206199   0.426595
  0.206199  0.0       0.0138327      -0.00198018   0.0206199   0.426595
 -0.134465  0.0       0.00110805      1.00781     -0.0134465  -0.0422246
  0.428032  0.0       0.233494       -0.0248687    1.0428      0.470084
  0.718619  0.0       0.185568       -0.0417518    0.0718619   1.28949

# **2. General approach**

First, we explain the basics of technosphere and biosphere exchanges.

In [14]:
SMR=getAct("hydrogen production, steam reforming","RoW")
lca(SMR)[end,:]


Row,Exchanges,Climate change-Energy imbalance,Climate change-CO2 Concentration,Ocean acidification-Carbonate ion concentration,Atmospheric aerosol loading-Aerosol Optical Depth (AOD),Freshwater use-Global,Biogeochemical flows-P,Biogeochemical flows-N,Stratospheric ozone depletion-Stratospheric O3 concentration,Land-system change-Global,Biosphere Integrity-Change in biosphere integrity
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
40,TOTAL,3.72281e-12,2.81667e-10,8.6058e-13,7.78278e-15,3.91042e-10,9.38267e-12,1.91231e-12,1.94085e-15,1.4544e-15,2.38269e-12


**Technosphere exchanges**
$$
\boldsymbol{As=f}
\tag{1}
$$


In [15]:
SMR


HTML{String}("<table style=\"border-collapse: collapse; width: 100%; font-family: Arial, sans-serif; font-size: 12px;\">\n    <thead>\n        <!-- Top heading row with dark grey background and white bold font -->\n        <tr style=\"background-color: #415a77; font-weight: bold; color: White;\">\n            <td rowspan=\"2\" style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                Activity key: <br><br>  20436\n            </td>\n            <td style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                Activity name\n            </td>\n            <td colspan=\"3\" style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                hydrogen production, steam reforming, RoW<br>\n            </td>\n        </tr>\n        <tr style=\"background-color: #415a77; font-weight: bold; color: white;\">\n            <td style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                Reference flow\n            </td>\n            <td colspan=\"3\" style=\"border: 1px solid #ddd; padding: 7px; text-align: center;\">\n                1.0 kilogram of hydrogen, gaseous\n            </td>\n        </tr>\n        <td colspan=\"5\" style=\"border: 1px solid #ddd; padding: 7px; background-color: #8d99ae; font-weight: bold; color: White; text-align: center;\">Technosphere Exchanges</td>\n        <!-- Column names row with centered text -->\n        \n    </thead>\n    <tbody>\n        <tr style=\"font-weight: bold;background-color: #8d99ae; padding: 7px; color: White; text-align: center\">\n            <td style=\"border: 1px solid #ddd; background-color: #8d99ae; font-weight: bold; color: White; text-align: center;\">Exchange keys</td>\n\n            <td style=\"border: 1px solid #ddd;\">Reference flow</td>\n            <td style=\"border: 1px solid #ddd;\">Activity</td>\n            <td style=\"border: 1px solid #ddd;\">Amount</td>\n            <td style=\"border: 1px solid #ddd;\">Unit</td>\n        </tr>\n    <tr style=\"font-size: 11px; padding: 3px;\">\n        <td style=\"border: 1px solid #ddd; \">443</td>\n        <td style=\"border: 1px solid #ddd;\">molybdenum trioxide</td>\n        <td style=\"border: 1px solid #ddd;\">market for molybdenum trioxide - GLO</td>\n        <td style=\"border: 1px solid #ddd;text-align: right\">1.668e-5</td>\n        <td style=\"border: 1px solid #ddd;text-align: left\">kilogram</td>\n    </tr>\n    <tr style=\"font-size: 11px; padding: 3px;\">\n        <td style=\"border: 1px solid #ddd; \">458</td>\n        <td style=\"border: 1px solid #ddd;\">chromium oxide, flakes</td>\n        <td style=\"border: 1px solid #ddd;\">market for chromium oxide, flakes - GLO</td>\n        <td style=\"border: 1px solid #ddd;text-align: right\">3.6e-5</td>\n        <td style=\"border: 1px solid #ddd;text-align: left\">kilogram</td>\n    </tr>\n    <tr style=\"font-size: 11px; padding: 3px;\">\n        <td style=\"border: 1px solid #ddd; \">795</td>\n        <td style=\"border: 1px solid #ddd;\">natural gas, high pressure</td>\n        <td style=\"border: 1px solid #ddd;\">market for natural gas, high pressure - JP</td>\n        <td style=\"border: 1px solid #ddd;text-align: right\">0.15582183696547777</td>\n        <td style=\"border: 1px solid #ddd;text-align: left\">cubic meter</td>\n    </tr>\n    <tr style=\"font-size: 11px; padding: 3px;\">\n        <td style=\"border: 1px solid #ddd; \">2029</td>\n        <td style=\"border: 1px solid #ddd;\">quicklime, milled, packed</td>\n        <td style=\"border: 1px solid #ddd;\">market for quicklime, milled, packed - RoW</td>\n        <td style=\"border: 1px solid #ddd;text-align: right\">4.8e-5</td>\n        <td style=\"border: 1px solid #ddd;text-align: left\">kilogram</td>\n    </tr>\n    <tr style=\"font-size: 11px; padding: 3px;\">\n        <td style=\"border: 1px solid #ddd; \">2521</td>\n        <td style=\"border: 1px solid #ddd;\">natural gas, high pressure</td>\n        <

Reference flow               Activity                     Amount       Unit     Location         Key             
──────────────────────────── ──────────────────────────── ──────────── ──────── ──────────────── ────────────────
hydrogen, gaseous            hydrogen production, steam reforming          1.0 kilogram RoW              20436           

In [16]:
𝐀=Technosphere!().Matrix


23559×23559 SparseMatrixCSC{Float64, Int64} with 291159 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡏⢸⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡯⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡧⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣯⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣧⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡧⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣽⣷⣺⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣹⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣗⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣇⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿⡿⢾⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣧⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡧⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣯⡯⢹⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣟⢻⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡏⢹⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⢯⡗⠊⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢯⎦

In [21]:
𝐀[:,SMR.key]


23559-element SparseVector{Float64, Int64} with 39 stored entries:
  [443  ]  =  -1.668e-5
  [458  ]  =  -3.6e-5
  [795  ]  =  -0.155822
  [2029 ]  =  -4.8e-5
  [2521 ]  =  -0.0634242
  [2580 ]  =  -0.0003624
  [2779 ]  =  -0.0158561
           ⋮
  [15775]  =  -0.00031236
  [16596]  =  -0.000532728
  [17191]  =  -0.090812
  [17195]  =  -0.000882902
  [17421]  =  -0.13074
  [17534]  =  -1.22034
  [19528]  =  -2.796e-5
  [20436]  =  1.0

In [22]:
𝐟 =zeros(𝐀.m)


23559-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [24]:
𝐟[SMR.key]=1 # kgH2/yr


1

In [26]:
sparse(𝐟)


23559-element SparseVector{Float64, Int64} with 1 stored entry:
  [20436]  =  1.0

In [27]:
𝐬=𝐀\𝐟



23559-element Vector{Float64}:
  5.377890524729087e-7
  3.5062077322241574e-12
  4.855325803316543e-12
  0.0
  0.0002539789356333746
  5.731000073263545e-10
  4.509626323112087e-8
  1.1169672015931043e-6
  0.0
 -0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

**Biosphere exchanges (total inventory)**

$$
\boldsymbol{Bs=BA^{-1}f=g}
\tag{2}
$$


In [28]:
𝐁=Biosphere!().Matrix


4709×23559 SparseMatrixCSC{Float64, Int64} with 437621 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣿⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎣⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⎦

In [29]:
𝐠 =𝐁*𝐬


4709-element Vector{Float64}:
 5.306669189174679e-15
 2.9004249171139055e-12
 0.0
 0.005478704778945987
 0.0004530910052546953
 0.0
 1.629495207438524e-9
 0.0
 0.0
 0.0
 ⋮
 1.1030998583665938e-11
 0.0
 4.248328108454661e-7
 0.0
 0.00030638420272426114
 2.9479616584594197e-13
 1.9027130478682404e-11
 0.0
 1.0396030498590655e-8

**Impact score calculation**
$$
\boldsymbol{QBs=QBA^{-1}f=h}
\tag{3}
$$

In [30]:
categories


10-element Vector{Any}:
 "Climate change-Energy imbalance"
 "Climate change-CO2 Concentration"
 "Ocean acidification-Carbonate ion concentration"
 "Atmospheric aerosol loading-Aerosol Optical Depth (AOD)"
 "Freshwater use-Global"
 "Biogeochemical flows-P"
 "Biogeochemical flows-N"
 "Stratospheric ozone depletion-Stratospheric O3 concentration"
 "Land-system change-Global"
 "Biosphere Integrity-Change in biosphere integrity"

In [32]:
𝐐=Characterisation!().Matrix


10×4709 SparseMatrixCSC{Float64, Int64} with 684 stored entries:
⎡⡿⡼⠯⠉⠭⠿⢈⡏⡇⢹⠿⠿⠤⢿⡸⠿⠅⠄⠸⠏⠿⠌⠿⠍⢨⠸⠏⢯⠇⠽⡼⠯⠯⠠⡿⠤⠇⢿⠏⢄⎤
⎣⣒⣖⣖⣒⣂⣐⣚⣒⣚⣰⣒⣕⣀⣚⡬⣂⣒⣂⣒⣐⣂⣓⣠⣒⣲⣂⣒⣚⣘⣒⣐⣲⣁⣒⣒⣐⣀⣚⣒⣂⎦

In [33]:
𝐡=𝐐*𝐠


10-element Vector{Float64}:
 3.722812863227291e-12
 2.8166669465216066e-10
 8.605802635455069e-13
 7.782779710254727e-15
 3.9104175936896077e-10
 9.382674532739888e-12
 1.912309790513869e-12
 1.9408514089273713e-15
 1.4544031457653483e-15
 2.3826881218181536e-12

**Absolute environmental sustainability assessment calculation**
$$
\boldsymbol{QBs\oslash\omega =QBA^{-1}f\oslash\omega =h\oslash\omega}
\tag{3}
$$

In [38]:
𝛚


10-element Vector{Float64}:
 1.5974315890692653e-14
 1.1501507441298711e-12
 1.099032933279655e-14
 1.118202112348486e-15
 6.389726356277062e-11
 1.757174747976192e-13
 9.904075852229445e-13
 2.2364042246969718e-13
 3.9935789726731633e-13
 1.5974315890692653e-13

In [39]:
𝐝 = 𝐡 ⊘ 𝛚
[a=>b for (a,b) ∈ zip(categories,𝐝)]


10-element Vector{Pair{String, Float64}}:
                              "Climate change-Energy imbalance" => 233.04990891011286
                             "Climate change-CO2 Concentration" => 244.89545921674056
              "Ocean acidification-Carbonate ion concentration" => 78.30340997857319
      "Atmospheric aerosol loading-Aerosol Optical Depth (AOD)" => 6.960083176653162
                                        "Freshwater use-Global" => 6.119851423446544
                                       "Biogeochemical flows-P" => 53.39636563492781
                                       "Biogeochemical flows-N" => 1.930831123515074
 "Stratospheric ozone depletion-Stratospheric O3 concentration" => 0.0086784463537237
                                    "Land-system change-Global" => 0.0036418539753874488
            "Biosphere Integrity-Change in biosphere integrity" => 14.915744361900428

Not very sustainable...


<div style="text-align:center;">
  <img src="./Images/PBI.svg" style="height:400px; display:inline-block;">
</div>


In [40]:
𝚪


10×10 Matrix{Float64}:
  1.22241   0.0      -0.0100732   …  -0.0710218    0.122241    0.38386
  0.0       1.22241  -0.0100732      -0.0710218    0.122241    0.38386
  0.325108  0.0       1.06115        -0.0188888    0.0325108   0.4567
 -0.684548  0.0       0.00564097      0.0397722   -0.0684548  -0.214962
  0.220814  0.0       0.0407336      -0.0128293    0.0220814   0.416858
  0.206199  0.0       0.0138327   …  -0.00198018   0.0206199   0.426595
  0.206199  0.0       0.0138327      -0.00198018   0.0206199   0.426595
 -0.134465  0.0       0.00110805      1.00781     -0.0134465  -0.0422246
  0.428032  0.0       0.233494       -0.0248687    1.0428      0.470084
  0.718619  0.0       0.185568       -0.0417518    0.0718619   1.28949

<div style="text-align:center;">
  <img src="./Images/interactions.svg" style="height:400px; display:inline-block;">
</div>


With interactions it's even worse...

In [42]:
𝐱.-𝐝


10-element Vector{Float64}:
   69.09960338501921
   71.7341373342673
   90.6462738934017
 -169.20372688527405
   63.095319232351585
   60.20778005882362
  111.67331457023636
  -33.23644635246455
  128.69247342297194
  193.57167973632681

In [41]:
𝐱 = 𝚪*𝐝
[a=>b for (a,b) ∈ zip(categories,𝐱)]


10-element Vector{Pair{String, Float64}}:
                              "Climate change-Energy imbalance" => 302.14951229513207
                             "Climate change-CO2 Concentration" => 316.62959655100786
              "Ocean acidification-Carbonate ion concentration" => 168.94968387197488
      "Atmospheric aerosol loading-Aerosol Optical Depth (AOD)" => -162.24364370862088
                                        "Freshwater use-Global" => 69.21517065579813
                                       "Biogeochemical flows-P" => 113.60414569375143
                                       "Biogeochemical flows-N" => 113.60414569375143
 "Stratospheric ozone depletion-Stratospheric O3 concentration" => -33.22776790611082
                                    "Land-system change-Global" => 128.69611527694732
            "Biosphere Integrity-Change in biosphere integrity" => 208.48742409822725

**Here, we implement the following model**

$$
\min_{\text{s.t.}\ s} \ \boldsymbol{x  = \Gamma d} 
\tag{1}
$$


$$
\boldsymbol{d = \left(\Lambda s\right)\oslash \omega = \left(Q B s\right)\oslash \omega}
\tag{2}
$$

$$
\boldsymbol{A^* s = f}
\tag{3}
$$

$$
\boldsymbol{s^j \geq 0}
\tag{4}
$$

$$
\boldsymbol{s^{el} \leq c^{el}}
\tag{5}
$$

In [ ]:
𝐀


23559×23559 SparseMatrixCSC{Float64, Int64} with 291159 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡏⢸⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡯⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡧⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣯⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣧⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡧⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣽⣷⣺⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣹⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣗⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣇⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿⡿⢾⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣧⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡧⢸⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣯⡯⢹⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⢼⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣟⢻⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡏⢹⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⢯⡗⠊⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢯⎦

<div style="text-align:center;">
  <img src="./Images/rectangularisation.svg" style="height:400px; display:inline-block;">
</div>


Okay now let's go to data preparation on activity browser and excel!

# **3.Modelling**


## **3.1 data**

## **3.2 Optimisation**

In [45]:
include("matrices.jl")


7-element Vector{String}:
 "Hydrogen (kg)"
 "Electricity (kWh)"
 "Water (kg)"
 "Units"
 "Units"
 "Units"
 "Units"

In [51]:
𝐀°=sparse(𝐀)


7×16 SparseMatrixCSC{Float64, Int64} with 22 stored entries:
⎡⠒⠒⠒⠒⠒⡀⠠⢸⎤
⎣⠀⠀⠀⠀⠀⠐⠄⣰⎦

In [48]:
processes


16-element Vector{String}:
 "electricity production, at\nwood" ⋯ 37 bytes ⋯ "igh voltage |\nRER | electricity"
 "electricity production, deep\nge" ⋯ 24 bytes ⋯ "high\nvoltage | SV | electricity"
 "electricity production,\nlignite" ⋯ 21 bytes ⋯ "voltage | US-NPCC |\nelectricity"
 "electricity production, wind,\n>" ⋯ 36 bytes ⋯ "high voltage | FR\n| electricity"
 "electricity production, hard\nco" ⋯ 37 bytes ⋯ "orth-eastern\ngrid | electricity"
 "electricity production,\nnatural" ⋯ 49 bytes ⋯ "h voltage | CN-SC |\nelectricity"
 "electricity production,\nphotovo" ⋯ 101 bytes ⋯ "d, multi-Si\n| ES | electricity"
 "electricity production,\nnuclear" ⋯ 63 bytes ⋯ "high voltage |\nAR | electricity"
 "electricity production, hydro,\n" ⋯ 31 bytes ⋯ "h voltage | IN-JK |\nelectricity"
 "electricity production, oil |\nelectricity, high voltage | CZ\n| electricity"
 "treatment of fuel cell stack,\n1" ⋯ 23 bytes ⋯ "ll\nstack, 1MWe, PEM | RER | PEM"
 "electrolyzer production, 1MWe,\n" ⋯ 24 bytes ⋯ "r,\

In [61]:
processes[scorrect]


14-element Vector{String}:
 "electricity production, at\nwood" ⋯ 37 bytes ⋯ "igh voltage |\nRER | electricity"
 "electricity production, deep\nge" ⋯ 24 bytes ⋯ "high\nvoltage | SV | electricity"
 "electricity production,\nlignite" ⋯ 21 bytes ⋯ "voltage | US-NPCC |\nelectricity"
 "electricity production, wind,\n>" ⋯ 36 bytes ⋯ "high voltage | FR\n| electricity"
 "electricity production, hard\nco" ⋯ 37 bytes ⋯ "orth-eastern\ngrid | electricity"
 "electricity production,\nnatural" ⋯ 49 bytes ⋯ "h voltage | CN-SC |\nelectricity"
 "electricity production,\nphotovo" ⋯ 101 bytes ⋯ "d, multi-Si\n| ES | electricity"
 "electricity production,\nnuclear" ⋯ 63 bytes ⋯ "high voltage |\nAR | electricity"
 "electricity production, hydro,\n" ⋯ 31 bytes ⋯ "h voltage | IN-JK |\nelectricity"
 "electricity production, oil |\nelectricity, high voltage | CZ\n| electricity"
 "electrolyzer production, 1MWe,\n" ⋯ 24 bytes ⋯ "r,\n1MWe, PEM, Stack | RER | PEM"
 "market for water, deionised |\nw" ⋯ 19 bytes ⋯ "uro

In [60]:
set_process=1:1:16
s_exclude=[11,13]
scorrect=setdiff(set_process,s_exclude)


14-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 12
 14
 15
 16

In [72]:
using Distributions


In [70]:
model = Model(CPLEX.Optimizer)
set_silent(model)

# Variables and expressions
@variable(model, 𝐬[1:length(processes)])
@expression(model, 𝐝, (𝐐*𝐁*𝐬) ⊘ 𝛚)
@expression(model, 𝐱, 𝐝)
@expression(model, φ, 𝐌.*𝐬) #costs tracks

@objective(model, Min, 𝐱);
@constraint(model, 𝐬[scorrect] ≥ 0)
@constraint(model, 𝐀° * 𝐬 == 𝐟)
@constraint(model, 𝐬[1:1:10] ≤ 𝐜)

optimize!(model);
if !is_solved_and_feasible(model)
    error("Solver did not find an optimal solution")
end

# # solution_summary(model)
# DataFrame(hcat([catnames_ticks,value.(𝐝),value.(𝐱)]...),["Boundaries","𝐝", "𝐱"])


In [100]:
𝐁°= cat(𝐁,𝐁,𝐁,𝐁,𝐁,dims=3)


209×16×5 Array{Float64, 3}:
[:, :, 1] =
 4.49428e-11  5.94707e-12  8.04452e-12  …  7.02653e-14     1.79002e-5   0.0
 1.83702e-13  1.88995e-13  1.33572e-13     3.41353e-15     8.77055e-7   0.0
 0.00147745   0.00335124   0.00335519      4.60885e-5   9169.48         0.0
 0.00059234   0.000516537  0.00047996      1.11284e-5   4784.98         0.0
 2.96067e-5   3.23197e-6   3.8832e-6       1.0594e-7      12.2551       0.0
 8.60272e-5   1.26184e-6   1.3414e-6    …  2.13793e-8      9.36411      0.0
 7.38571e-11  4.47138e-11  1.70896e-10     1.97961e-9      0.000846283  0.0
 6.96439e-13  7.16504e-13  5.06389e-13     1.29411e-14     3.32503e-6   0.0
 5.45637e-13  5.58344e-13  5.7975e-13      1.50582e-14     2.81552e-6   0.0
 6.6477e-12   1.67008e-11  9.39647e-12     3.07447e-13     4.04816e-5   0.0
 ⋮                                      ⋱                               ⋮
 8.29672e-6   1.16235e-5   7.70525e-5   …  3.17639e-8     54.187        0.0
 3.44263e-7   7.55553e-8   7.3159e-8       1.9025e

In [101]:
s°=zeros(length(processes),size(𝐁°,3))


16×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [ ]:
for (ind,b) ∈ enumerate(1:1:5)

    b=𝐁°[:,:,ind]
    model = Model(CPLEX.Optimizer)
    set_silent(model)

    # Variables and expressions
    @variable(model, 𝐬[1:length(processes)])
    @expression(model, 𝐝, (𝐐*b*𝐬) ⊘ 𝛚)
    @expression(model, 𝐱, 𝐝)
    @expression(model, φ, 𝐌.*𝐬) #costs tracking


    @objective(model, Min, 𝐱);
    @constraint(model, 𝐬[scorrect] ≥ 0)
    @constraint(model, 𝐀° * 𝐬 == 𝐟)
    @constraint(model, 𝐬[1:1:10] ≤ 𝐜)

    optimize!(model);
    if !is_solved_and_feasible(model)
        error("Solver did not find an optimal solution")
    end

    s°[:,ind] = value.(𝐬)
end


In [104]:
s°


16×5 Matrix{Float64}:
  0.0       0.0       0.0       0.0       0.0
  7.0       7.0       7.0       7.0       7.0
  6.0       6.0       6.0       6.0       6.0
  7.0       7.0       7.0       7.0       7.0
  7.0       7.0       7.0       7.0       7.0
  7.0       7.0       7.0       7.0       7.0
  7.0       7.0       7.0       7.0       7.0
  7.0       7.0       7.0       7.0       7.0
  7.0       7.0       7.0       7.0       7.0
  0.0       0.0       0.0       0.0       0.0
 -1.35e-6  -1.35e-6  -1.35e-6  -1.35e-6  -1.35e-6
  1.35e-6   1.35e-6   1.35e-6   1.35e-6   1.35e-6
 -3.37e-7  -3.37e-7  -3.37e-7  -3.37e-7  -3.37e-7
 14.0      14.0      14.0      14.0      14.0
  3.37e-7   3.37e-7   3.37e-7   3.37e-7   3.37e-7
  1.0       1.0       1.0       1.0       1.0